In [ ]:
## 初始化程式, 自己設 PREFIX 的路徑
import os
WORK_DIR = "/home/sean/yolov3_tensorflow"
%cd {WORK_DIR}

SUPERVISOR_PROJECT_NAME = "CAM1-24-4min-image"

SUPERVISOR_VIDEO_NAME = "CAM1-24-4min.mp4"

!conda env list

In [ ]:
## 下載 OID, OID 可以拿已經標記好的圖片
!git clone https://github.com/theAIGuysCode/OIDv4_ToolKit.git
## 安裝 OIDv4_ToolKit 下載圖片所需要的模組
%cd {WORK_DIR}/OIDv4_ToolKit/
%pip install -r ./requirements.txt

In [ ]:
## 刪除已經下載的圖片及標記
!rm -rf {WORK_DIR}/OIDv4_ToolKit/OID/Dataset/

In [ ]:
## 下載圖片
!python {WORK_DIR}/OIDv4_ToolKit/main.py downloader --classes Person --type_csv train --multiclasses 1 --limit 10 -y
!python {WORK_DIR}/OIDv4_ToolKit/main.py downloader --classes Person --type_csv test --multiclasses 1 --limit 1 -y

In [ ]:
#下載 darknet, darknet 可以用 yolov3 模型訓練及測試
%cd {WORK_DIR}
!git clone https://github.com/AlexeyAB/darknet.git

這邊在自己的電腦做
STEP 1: 下載影片
STEP 2: 利用 supervisely 標記圖片
STEP 3: 匯出標記好的 .json 和 image 會是 .rar 
STEP 4: 將 .rar 解壓縮 **到工作目錄**
STEP 5: 利用 video_split_convert.py 將 `project_name`/`video_name`/img 轉換成 yolov3 所需要的格式

In [ ]:
## 利用 video_split_convert.py 將 `project_name`/`video_name`/img 轉換成 yolov3 所需要的格式
%cd {WORK_DIR}/{SUPERVISOR_PROJECT_NAME}/{SUPERVISOR_VIDEO_NAME}
!cp -f video_split_convert.py {WORK_DIR}/{SUPERVISOR_PROJECT_NAME}/{SUPERVISOR_VIDEO_NAME}/
!python {WORK_DIR}/{SUPERVISOR_PROJECT_NAME}/{SUPERVISOR_VIDEO_NAME}/video_split_convert.py

In [ ]:
## 把照片的資訊放到 train.txt 及 test.txt !!!warning 這邊可能可以改進 
%cd {WORK_DIR}/{SUPERVISOR_PROJECT_NAME}/{SUPERVISOR_VIDEO_NAME}/img/

!ls -d -1 "$PWD/"*.* | grep png > train.txt
!ls -d -1 "$PWD/"*.* | grep png > test.txt
!mv ./train.txt {WORK_DIR}/darknet/build/darknet/x64/data/train.txt
!mv ./test.txt {WORK_DIR}/darknet/build/darknet/x64/data/test.txt

In [ ]:
## 更改 yolo 模型參數
%cd {WORK_DIR}/darknet/
!cp cfg/yolov3-tiny.cfg cfg/yolo-obj.cfg

!sed -i "s/batch=1/batch=32/" cfg/yolo-obj.cfg
!sed -i "s/subdivisions=1/subdivisions=8/" cfg/yolo-obj.cfg
!sed -i "s/max_batches = 500200/max_batches = 10000/" cfg/yolo-obj.cfg
!sed -i "s/steps=400000,450000/steps=800,900/" cfg/yolo-obj.cfg
!sed -i "s/classes=80/classes=1/g" cfg/yolo-obj.cfg
# (class + 5) * 3
!sed -i "s/filters=255/filters=18/g" cfg/yolo-obj.cfg 
!sed -i "s/mosaic=1/mosaic=0/g" cfg/yolo-obj.cfg

In [ ]:
## 宣告有什麼 class 及 各種檔案路徑
!printf "person" > {WORK_DIR}/darknet/build/darknet/x64/data/obj.names
!printf "classes = 1\ntrain  = {WORK_DIR}/darknet/build/darknet/x64/data/train.txt\nvalid  = {WORK_DIR}/darknet/build/darknet/x64/data/test.txt\nnames = {WORK_DIR}/darknet/build/darknet/x64/data/obj.names\nbackup = backup/" > {WORK_DIR}/darknet/build/darknet/x64/data/obj.data

In [ ]:
## 將圖片放到指定路徑
!rm -rf {WORK_DIR}/darknet/build/darknet/x64/data/obj/
!mkdir -p {WORK_DIR}/darknet/build/darknet/x64/data/obj/
!cp -r {WORK_DIR}/OIDv4_ToolKit/OID/Dataset/train/Person/* {WORK_DIR}/darknet/build/darknet/x64/data/obj/
!cp -r {WORK_DIR}/OIDv4_ToolKit/OID/Dataset/test/Person/* {WORK_DIR}/darknet/build/darknet/x64/data/obj/

In [ ]:
## 從預訓練參數開始訓練
!cp {WORK_DIR}/yolov3-tiny.conv.11 {WORK_DIR}/darknet/build/darknet/x64/

In [ ]:
## 更改 darknet 的參數
%cd {WORK_DIR}/darknet/
!sed -i "s/GPU=0/GPU=1/" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/" Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/" Makefile
!sed -i "s/OPENCV=1/OPENCV=0/" Makefile
!make 

In [ ]:
%cd {WORK_DIR}/darknet/
!./darknet detector train {WORK_DIR}/darknet/build/darknet/x64/data/obj.data {WORK_DIR}/darknet/cfg/yolo-obj.cfg {WORK_DIR}/yolov3-tiny.conv.11